In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm

In [37]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [8]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
sys.path.append('../../../../nadavp/opt_lev_analysis/scripts/calibration/')
import BeadDataFile
from discharge_tools import *

In [9]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [48]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [297]:
##example
bu1 = bu.DataFile()
bu1.load('/data/old_trap/20210203/gbead3/neutrality_1/10V_10000s_no_freq_comb/turbombar_powfb_xyzcool_10.h5')
bu1.load_other_data()

In [18]:
step_cal_dir = r'/data/old_trap/20210203/gbead3/discharge/fine/'
max_file = 500

In [109]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)

In [292]:
# Do the step calibration
step_file_objs = []
for filname in step_cal_files[1:max_file]:
    df = bu.DataFile();
    df.load(filname);
    df.load_other_data()
    step_file_objs.append(df);

In [299]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 41
def func1_scale(i):
    bb = step_file_objs[i]
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [300]:
func1_scale(0)

3438.2412934040485

In [223]:
amps3 = []
for i in range(332):
    print(i,'/332')
    amps3.append(func1_scale(i))

0 /332
1 /332
2 /332
3 /332
4 /332
5 /332
6 /332
7 /332
8 /332
9 /332
10 /332
11 /332
12 /332
13 /332
14 /332
15 /332
16 /332
17 /332
18 /332
19 /332
20 /332
21 /332
22 /332
23 /332
24 /332
25 /332
26 /332
27 /332
28 /332
29 /332
30 /332
31 /332
32 /332
33 /332
34 /332
35 /332
36 /332
37 /332
38 /332
39 /332
40 /332
41 /332
42 /332
43 /332
44 /332
45 /332
46 /332
47 /332
48 /332
49 /332
50 /332
51 /332
52 /332
53 /332
54 /332
55 /332
56 /332
57 /332
58 /332
59 /332
60 /332
61 /332
62 /332
63 /332
64 /332
65 /332
66 /332
67 /332
68 /332
69 /332
70 /332
71 /332
72 /332
73 /332
74 /332
75 /332
76 /332
77 /332
78 /332
79 /332
80 /332
81 /332
82 /332
83 /332
84 /332
85 /332
86 /332
87 /332
88 /332
89 /332
90 /332
91 /332
92 /332
93 /332
94 /332
95 /332
96 /332
97 /332
98 /332
99 /332
100 /332
101 /332
102 /332
103 /332
104 /332
105 /332
106 /332
107 /332
108 /332
109 /332
110 /332
111 /332
112 /332
113 /332
114 /332
115 /332
116 /332
117 /332
118 /332
119 /332
120 /332
121 /332
122 /332
123

In [207]:
_,ax = plt.subplots(figsize=(9.5,4))
ax.scatter(range(332), phases2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [301]:
amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
scale = 145

# Perrinf
ax.scatter(range(len(amps)), amps/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.legend()
ax.set(xlabel='dataset', ylabel='Charges', ylim=(-4.5,0.3), xlim=(205,333))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [232]:
scale = np.abs(np.mean(amps3[280:310]))
print("Scale is ", scale)

Scale is  142.85646989


### No phase response

In [316]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 43
def get_amp(i):
    
#     print(i,'  ******************')
    fname = r'/data/old_trap/20210203/gbead3/neutrality_1/10V_10000s/turbombar_powfb_xyzcool_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.electrode_data[3])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    phi_tmp = m1_tmp.values[2]
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1000, 10000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [317]:
amps_10V2 = []
for i in range(1000):
    print(i,'/5000')
    if i!=1479:
        amps_10V2.append(get_amp(i))

0 /5000
1 /5000
2 /5000
3 /5000
4 /5000
5 /5000
6 /5000
7 /5000
8 /5000
9 /5000
10 /5000
11 /5000
12 /5000
13 /5000
14 /5000
15 /5000
16 /5000
17 /5000
18 /5000
19 /5000
20 /5000
21 /5000
22 /5000
23 /5000
24 /5000
25 /5000
26 /5000
27 /5000
28 /5000
29 /5000
30 /5000
31 /5000
32 /5000
33 /5000
34 /5000
35 /5000
36 /5000
37 /5000
38 /5000
39 /5000
40 /5000
41 /5000
42 /5000
43 /5000
44 /5000
45 /5000
46 /5000
47 /5000
48 /5000
49 /5000
50 /5000
51 /5000
52 /5000
53 /5000
54 /5000
55 /5000
56 /5000
57 /5000
58 /5000
59 /5000
60 /5000
61 /5000
62 /5000
63 /5000
64 /5000
65 /5000
66 /5000
67 /5000
68 /5000
69 /5000
70 /5000
71 /5000
72 /5000
73 /5000
74 /5000
75 /5000
76 /5000
77 /5000
78 /5000
79 /5000
80 /5000
81 /5000
82 /5000
83 /5000
84 /5000
85 /5000
86 /5000
87 /5000
88 /5000
89 /5000
90 /5000
91 /5000
92 /5000
93 /5000
94 /5000
95 /5000
96 /5000
97 /5000
98 /5000
99 /5000
100 /5000
101 /5000
102 /5000
103 /5000
104 /5000
105 /5000
106 /5000
107 /5000
108 /5000
109 /5000
110 /5000


832 /5000
833 /5000
834 /5000
835 /5000
836 /5000
837 /5000
838 /5000
839 /5000
840 /5000
841 /5000
842 /5000
843 /5000
844 /5000
845 /5000
846 /5000
847 /5000
848 /5000
849 /5000
850 /5000
851 /5000
852 /5000
853 /5000
854 /5000
855 /5000
856 /5000
857 /5000
858 /5000
859 /5000
860 /5000
861 /5000
862 /5000
863 /5000
864 /5000
865 /5000
866 /5000
867 /5000
868 /5000
869 /5000
870 /5000
871 /5000
872 /5000
873 /5000
874 /5000
875 /5000
876 /5000
877 /5000
878 /5000
879 /5000
880 /5000
881 /5000
882 /5000
883 /5000
884 /5000
885 /5000
886 /5000
887 /5000
888 /5000
889 /5000
890 /5000
891 /5000
892 /5000
893 /5000
894 /5000
895 /5000
896 /5000
897 /5000
898 /5000
899 /5000
900 /5000
901 /5000
902 /5000
903 /5000
904 /5000
905 /5000
906 /5000
907 /5000
908 /5000
909 /5000
910 /5000
911 /5000
912 /5000
913 /5000
914 /5000
915 /5000
916 /5000
917 /5000
918 /5000
919 /5000
920 /5000
921 /5000
922 /5000
923 /5000
924 /5000
925 /5000
926 /5000
927 /5000
928 /5000
929 /5000
930 /5000
931 /5000


In [274]:
amps_50V = np.array(amps_50V)
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(amps_50V)), amps_50V/scale, label='50V, 41Hz., NoSpin')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [275]:
_,ax = plt.subplots()
data = amps_50V/scale
_ = ax.hist(data, bins=50, range=(-0.5,0.5))
ax.set(xlabel=r'$\epsilon$')
np.mean(data), np.std(data)/np.sqrt(len(data))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0065643952062581978, 0.0020025317881265784)

In [278]:
data = amps_50V/scale
means = [np.mean(data[j*1000:(j+1)*1000]) for j in range(5)]
stds = [np.std(data[j*1000:(j+1)*1000]) for j in range(5)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(np.arange(0,5000,1000), means, yerr=stds/np.sqrt(1000),fmt='o')
ax.plot(np.arange(0,5000,1000), np.arange(0,5000,1000)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'dataset'), Text(0, 0.5, '$\\epsilon$')]

In [374]:
data = np.array(amps_10V)/scale*5
means = [np.mean(data[j*1000:(j+1)*1000]) for j in range(1)]
stds = [np.std(data[j*1000:(j+1)*1000]) for j in range(1)]

data2 = np.array(amps_10V2)/scale*30
means2 = [np.mean(data2[j*1000:(j+1)*1000]) for j in range(1)]
stds2 = [np.std(data2[j*1000:(j+1)*1000]) for j in range(1)]

means3 = [np.mean(data3[j*1000:(j+1)*1000]) for j in range(1)]
stds3 = [np.std(data3[j*1000:(j+1)*1000]) for j in range(1)]

_,ax = plt.subplots(figsize=(9.5,4))
ax.errorbar(1, means, yerr=stds/np.sqrt(1000),fmt='o', label='10V - single frequency')
ax.errorbar(2, means2, yerr=stds2/np.sqrt(1000),fmt='o', label='10V - single frequency from a comb')
ax.errorbar(3, means3, yerr=stds3/np.sqrt(1000),fmt='o', label='10V - frequency comb')
ax.plot(np.arange(0,4,0.2), np.arange(0,4,0.2)*0, 'r--')
ax.set(xlabel='dataset', ylabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### MultiHarmonic

In [333]:
np.ones(len(neutrality_freqs_all))

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [351]:
neutrality_scales, neutrality_phases

(array([ 145.]), array([[ 6.04291893]]))

In [371]:
# likelihood calculator stuff
neutrality_freqs_all = np.array([5 ,7, 9, 13, 17, 23, 29, 37, 43, 51, 57, 69, 71])
neutrality_freqs_all = np.array([7, 9, 13, 17, 23, 29, 37, 43, 51, 57, 69, 71])

MLEs_43Hz = []
for i in range(1000):
    print(i,'  ******************')
    fname = r'/data/old_trap/20210203/gbead3/neutrality_1/10V_10000s/turbombar_powfb_xyzcool_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20210203/gbead3/discharge/fine/turbombar_powfb_xyzcool_elec3_2000mV41Hz0mVdc_'+str(i)+'.h5'
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()   
    
    x = bb.electrode_data[3]
    freq = np.fft.rfftfreq(len(x), d=1./fsamp)
    angles = np.angle(np.fft.rfft(x))
    phase_all = np.array([angles[freq_==freq] for freq_ in neutrality_freqs_all])
    neutrality_phases = (phase_all+np.pi/2)%(2*np.pi)
    neutrality_scales = np.ones(len(phase_all))*scale ## flat signal model as no TF exsit
    noises_std = np.ones(len(phase_all))

    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':3000,
          'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
          'limit_phi': [0, np.pi],
          'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
          'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    mm_tmp = ll.find_mle_multiHarmoincs_flat_template(x=bb.pos_data[0], scales=neutrality_scales/30, phases=neutrality_phases, 
                                          signal_freqs=neutrality_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    MLEs_43Hz.append(mm_tmp.values[0])

0   ******************
MLEs:  <ValueView of Minuit at 14dad0e98>
  A: -0.4421935131540522
  phi: 0.0
  sigma: 4915.768866529169
1   ******************
MLEs:  <ValueView of Minuit at efa41928>
  A: 0.1422362317534862
  phi: 0.0
  sigma: 3344.8096030677666
2   ******************
MLEs:  <ValueView of Minuit at ebda43f8>
  A: 1.8856046708096983
  phi: 0.0
  sigma: 3682.050825673633
3   ******************
MLEs:  <ValueView of Minuit at efa41928>
  A: 0.10155533489887603
  phi: 0.0
  sigma: 2826.732062119586
4   ******************
MLEs:  <ValueView of Minuit at 14dad0e98>
  A: -0.7668729388824431
  phi: 0.0
  sigma: 1980.0331423995842
5   ******************
MLEs:  <ValueView of Minuit at 15a9243b8>
  A: 0.34934186028840486
  phi: 0.0
  sigma: 3323.2689310156993
6   ******************
MLEs:  <ValueView of Minuit at e776ed78>
  A: -0.30284999607829377
  phi: 0.0
  sigma: 2531.5795264886365
7   ******************
MLEs:  <ValueView of Minuit at 135714178>
  A: 0.18291317684634123
  phi: 0.0
  si

MLEs:  <ValueView of Minuit at 142df42a8>
  A: 0.07143653779348824
  phi: 0.0
  sigma: 4680.140071497085
65   ******************
MLEs:  <ValueView of Minuit at 12f1449b8>
  A: -0.16420281985483598
  phi: 0.0
  sigma: 3514.7511682563913
66   ******************
MLEs:  <ValueView of Minuit at 134e1e338>
  A: 1.4415185362886405
  phi: 0.0
  sigma: 3218.459207951663
67   ******************
MLEs:  <ValueView of Minuit at fd3e7cc8>
  A: -0.2486771918775048
  phi: 0.0
  sigma: 3810.3243380914196
68   ******************
MLEs:  <ValueView of Minuit at 10615a2c8>
  A: -0.7705671500880271
  phi: 0.0
  sigma: 3665.455002513529
69   ******************
MLEs:  <ValueView of Minuit at 16014dfb8>
  A: -0.08214827803021763
  phi: 0.0
  sigma: 3089.919952734388
70   ******************
MLEs:  <ValueView of Minuit at 1117b6738>
  A: -0.7582839246169897
  phi: 0.0
  sigma: 2134.676831225202
71   ******************
MLEs:  <ValueView of Minuit at 1117b7d28>
  A: 1.4325748161791125
  phi: 0.0
  sigma: 3953.1716

MLEs:  <ValueView of Minuit at 15a75cff8>
  A: 0.23462770569312852
  phi: 0.0
  sigma: 3677.738793943286
129   ******************
MLEs:  <ValueView of Minuit at d8c29348>
  A: 0.8588437377475202
  phi: 0.0
  sigma: 4003.76894196397
130   ******************
MLEs:  <ValueView of Minuit at 10cd4a018>
  A: -1.42511322577775
  phi: 0.0
  sigma: 5026.257758747488
131   ******************
MLEs:  <ValueView of Minuit at 11393a2b8>
  A: -0.33759153696883004
  phi: 0.0
  sigma: 3430.8123350637597
132   ******************
MLEs:  <ValueView of Minuit at 10615a568>
  A: -7.765597547404468e-05
  phi: 0.0
  sigma: 0.7929434989728534
133   ******************
MLEs:  <ValueView of Minuit at d45bb558>
  A: -0.7083553626289358
  phi: 0.0
  sigma: 2665.6128743683394
134   ******************
MLEs:  <ValueView of Minuit at fd3e7fb8>
  A: -1.2024286083324114
  phi: 0.0
  sigma: 4228.336120439546
135   ******************
MLEs:  <ValueView of Minuit at fd3e77d8>
  A: -0.07066453275911044
  phi: 0.0
  sigma: 489

MLEs:  <ValueView of Minuit at 11393ae08>
  A: 0.9981885251036147
  phi: 0.0
  sigma: 4540.6412618570475
193   ******************
MLEs:  <ValueView of Minuit at ec611598>
  A: -0.21625927225977648
  phi: 0.0
  sigma: 2285.2543262280587
194   ******************
MLEs:  <ValueView of Minuit at ec6111f8>
  A: 0.14019144779013004
  phi: 0.0
  sigma: 2380.284063088434
195   ******************
MLEs:  <ValueView of Minuit at 142df32b8>
  A: 0.3119503532652743
  phi: 0.0
  sigma: 4574.440466673824
196   ******************
MLEs:  <ValueView of Minuit at 113939f18>
  A: 0.9039686385367531
  phi: 0.0
  sigma: 4211.6087589746385
197   ******************
MLEs:  <ValueView of Minuit at 2db83758>
  A: 0.6021643883141223
  phi: 0.0
  sigma: 1975.7064143633984
198   ******************
MLEs:  <ValueView of Minuit at 151771778>
  A: -0.6629795785702299
  phi: 0.0
  sigma: 2142.9678014829865
199   ******************
MLEs:  <ValueView of Minuit at d8c28668>
  A: 0.4418598136107903
  phi: 0.0
  sigma: 3536.7

MLEs:  <ValueView of Minuit at 15a9243b8>
  A: 0.07206166065589059
  phi: 0.0
  sigma: 4726.895731615499
257   ******************
MLEs:  <ValueView of Minuit at 14dad0e98>
  A: 1.4028826276480686
  phi: 0.0
  sigma: 5118.586873821163
258   ******************
MLEs:  <ValueView of Minuit at efa41928>
  A: -0.03620197717100382
  phi: 0.0
  sigma: 3051.8224129497116
259   ******************
MLEs:  <ValueView of Minuit at fd3e7b38>
  A: -0.2146628728369251
  phi: 0.0
  sigma: 3515.302607539128
260   ******************
MLEs:  <ValueView of Minuit at 159561388>
  A: -0.9329301363177365
  phi: 0.0
  sigma: 2544.4206654430814
261   ******************
MLEs:  <ValueView of Minuit at 15955fda8>
  A: 0.42346252384595573
  phi: 0.0
  sigma: 4442.196222219197
262   ******************
MLEs:  <ValueView of Minuit at d45bc428>
  A: -0.866654559969902
  phi: 0.0
  sigma: 3417.7203026872216
263   ******************
MLEs:  <ValueView of Minuit at d45bc0f8>
  A: -0.2354510082222987
  phi: 0.0
  sigma: 2765.

...

MLEs:  <ValueView of Minuit at de833e08>
  A: 0.4776288191205822
  phi: 0.0
  sigma: 2321.0906718287292
705   ******************
MLEs:  <ValueView of Minuit at d45bbdf8>
  A: -1.0722361736552557
  phi: 0.0
  sigma: 4022.454503371242
706   ******************
MLEs:  <ValueView of Minuit at de833038>
  A: 0.414672061757301
  phi: 0.0
  sigma: 3960.857595148748
707   ******************
MLEs:  <ValueView of Minuit at 1759791f8>
  A: 1.4891389430704294
  phi: 0.0
  sigma: 3178.485965045108
708   ******************
MLEs:  <ValueView of Minuit at 180c534d8>
  A: -0.47728172229835764
  phi: 0.0
  sigma: 2799.534181020302
709   ******************
MLEs:  <ValueView of Minuit at 16014f2c8>
  A: -2.857637247914681
  phi: 0.0
  sigma: 5609.02925483786
710   ******************
MLEs:  <ValueView of Minuit at 15296bf18>
  A: 1.8872552725515561
  phi: 0.0
  sigma: 5167.517243074071
711   ******************
MLEs:  <ValueView of Minuit at 15a918168>
  A: 0.233320473213098
  phi: 0.0
  sigma: 3703.22387203

MLEs:  <ValueView of Minuit at efa41928>
  A: 0.7377785238059005
  phi: 0.0
  sigma: 1825.6354643438333
769   ******************
MLEs:  <ValueView of Minuit at 10615a0d8>
  A: -0.6670896986033767
  phi: 0.0
  sigma: 1950.4108408968102
770   ******************
MLEs:  <ValueView of Minuit at ec6111f8>
  A: 1.619039733108366
  phi: 0.0
  sigma: 4612.236708096185
771   ******************
MLEs:  <ValueView of Minuit at 15955fff8>
  A: -0.4828836883971235
  phi: 0.0
  sigma: 2363.853355850931
772   ******************
MLEs:  <ValueView of Minuit at 15bb1a298>
  A: -0.3440027355100028
  phi: 0.0
  sigma: 2395.4051117186514
773   ******************
MLEs:  <ValueView of Minuit at 1117b85c8>
  A: 0.13877528606099077
  phi: 0.0
  sigma: 1801.61807333723
774   ******************
MLEs:  <ValueView of Minuit at 180c53be8>
  A: -0.7177751100971363
  phi: 0.0
  sigma: 3474.763060861419
775   ******************
MLEs:  <ValueView of Minuit at 175979e68>
  A: 0.3487492510321317
  phi: 0.0
  sigma: 3216.12

MLEs:  <ValueView of Minuit at 134e1db28>
  A: -0.4120962953165872
  phi: 0.0
  sigma: 2970.881288830218
833   ******************
MLEs:  <ValueView of Minuit at 142df3ad8>
  A: -0.13845099456375465
  phi: 0.0
  sigma: 3560.17044624668
834   ******************
MLEs:  <ValueView of Minuit at 1075d0d78>
  A: 0.03269888466456905
  phi: 0.0
  sigma: 2091.0706635918764
835   ******************
MLEs:  <ValueView of Minuit at ebda43f8>
  A: -0.8084621536399936
  phi: 0.0
  sigma: 3213.7903719693386
836   ******************
MLEs:  <ValueView of Minuit at 135714178>
  A: 1.0774576448311564
  phi: 0.0
  sigma: 4444.926077721733
837   ******************
MLEs:  <ValueView of Minuit at 14dad0e98>
  A: 0.7387256274960237
  phi: 0.0
  sigma: 1891.1960388307002
838   ******************
MLEs:  <ValueView of Minuit at efa41928>
  A: 0.1887197000469314
  phi: 0.0
  sigma: 3294.228479521488
839   ******************
MLEs:  <ValueView of Minuit at ec6111f8>
  A: 1.598636014634394
  phi: 0.0
  sigma: 2774.888

MLEs:  <ValueView of Minuit at 15a75cd78>
  A: -0.22577078598260414
  phi: 0.0
  sigma: 3224.881970029514
897   ******************
MLEs:  <ValueView of Minuit at 15bb1b178>
  A: 2.056113173661288
  phi: 0.0
  sigma: 4756.5736462129025
898   ******************
MLEs:  <ValueView of Minuit at fd3e6f08>
  A: -0.22703580890083686
  phi: 0.0
  sigma: 2835.3673889837673
899   ******************
MLEs:  <ValueView of Minuit at 10bdc96d8>
  A: 0.7129725377599243
  phi: 0.0
  sigma: 3691.862250525868
900   ******************
MLEs:  <ValueView of Minuit at 184886248>
  A: 1.3296638261526823
  phi: 0.0
  sigma: 3019.6263438470596
901   ******************
MLEs:  <ValueView of Minuit at 1587b428>
  A: 0.9374878811504459
  phi: 0.0
  sigma: 3073.9507867796888
902   ******************
MLEs:  <ValueView of Minuit at 1075d16b8>
  A: -0.28688364625850227
  phi: 0.0
  sigma: 3410.584963398744
903   ******************
MLEs:  <ValueView of Minuit at 157164c38>
  A: -1.3391115966660436
  phi: 0.0
  sigma: 337

MLEs:  <ValueView of Minuit at d45ba8b8>
  A: 0.34309577381645795
  phi: 0.0
  sigma: 3855.5080302906663
961   ******************
MLEs:  <ValueView of Minuit at 184886568>
  A: 0.728301041162922
  phi: 0.0
  sigma: 3445.427414765945
962   ******************
MLEs:  <ValueView of Minuit at 106159158>
  A: -0.43110247669392265
  phi: 0.0
  sigma: 2666.1469856889553
963   ******************
MLEs:  <ValueView of Minuit at 180c54638>
  A: 1.6452997578307986
  phi: 0.0
  sigma: 3776.40843956345
964   ******************
MLEs:  <ValueView of Minuit at 15955fe48>
  A: -0.6061663425498409
  phi: 0.0
  sigma: 4709.218019174732
965   ******************
MLEs:  <ValueView of Minuit at 15a919878>
  A: -0.35025537690671626
  phi: 0.0
  sigma: 2212.611889611915
966   ******************
MLEs:  <ValueView of Minuit at de830c68>
  A: 2.0707433906645747
  phi: 0.0
  sigma: 4424.766352096432
967   ******************
MLEs:  <ValueView of Minuit at 1075d12d8>
  A: 0.25603423654683866
  phi: 0.0
  sigma: 3164.8

In [373]:
np.mean(data3), np.std(data3)/np.sqrt(1000)

(-0.059762144060863651, 0.031428466008481454)

In [372]:
_,ax = plt.subplots()
data = np.array(amps_10V)/scale*5
data2 = np.array(amps_10V2)/scale*30
data3 = np.array(MLEs_43Hz)

_ = ax.hist(data, bins=50, range=(-10,10))
_ = ax.hist(data2, bins=50, range=(-10,10))
_ = ax.hist(data3, bins=50, range=(-10,10))
ax.set(xlabel=r'$\epsilon$')
np.mean(data), np.std(data)/np.sqrt(len(data))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0060662657244999951, 0.014382483568005635)

In [302]:
##10V frequency comb
bu1 = bu.DataFile()
bu1.load('/data/old_trap/20210203/gbead3/neutrality_1/10V_10000s/turbombar_powfb_xyzcool_10.h5')
bu1.load_other_data()

In [328]:
angles[freq==43]

array([-1.81106307])

In [364]:
x = bu1.pos_data[0]
# x = bu1.other_data[2,:]*-100
# x = bu1.electrode_data[3]
fsamp = 5000
fft = np.abs(np.fft.rfft(x,norm='ortho'))**2
angles = np.angle(np.fft.rfft(x,norm='ortho'))
freq = np.fft.rfftfreq(len(x), d=1./fsamp)
_, ax= plt.subplots()
ax.semilogy(freq, fft)
ax.set(xlim=(0,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 100.0)]

In [315]:
45/1.5

30.0

In [313]:
_,ax = plt.subplots()
x = bu1.other_data[4,:]*-100
x = bu1.electrode_data[3]
bandwidth = 1
drive_freq = 43
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
responsefilt = signal.filtfilt(b, a, x)
ax.scatter(range(len(x)),responsefilt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
[np.std(bu1.electrode_data[i]) for i in range(3)]

[0.0015941114334470346, 0.0014892385580929, 0.0014782817850800825]

In [10]:
def load_dir2(dirname, file_prefix = 'Discharge', start_file=0, max_file=500):
    ''' Load all files in directory to a list of BeadDataFile
        INPUTS: dirname, directory name
        max_file, maximum number of files to read'''

    ## Load all filenames in directory
    files = []
    [files.append(file_) for file_ in os.listdir(dirname) if file_.startswith(file_prefix) if file_.endswith('.h5')]
    files.sort(key=lambda ff: int(os.path.splitext(ff)[0].split('_')[-1])) ## sort by index

    # Load data into a BeadDataFile list
    BDFs = [BDF.BeadDataFile(dirname+filename) for filename in files[start_file:start_file+max_file]]
    print(len(files),' files in folder')
    print(len(BDFs),' files loaded')

    return BDFs


ImportError: constructor returned NULL